## Step 1: YOLO to COCO Conversion
### Configuration and Initialization
    

Set up the data directory path and define your class mapping for the object detection task. 
            - Update `DATA_DIR` to point to your dataset folder containing `.jpg` images and `.txt` YOLO annotations.
            - Modify `CLASS_MAPPING` to match your dataset's class IDs and names

In [ ]:
import os
import json
import glob
from PIL import Image
from datetime import datetime

# Configuration: Data directory path
# DATA_DIR = '/dataset/path'  # Parent directory containing images and annotations

# Boats orientation class mapping
CLASS_MAPPING = {
    0: 'Small Craft', 1: 'Small Fishing Boat', 2: 'Small Passenger Ship',
    3: 'Fishing Trawler', 4: 'Large Passenger Ship', 5: 'Sailing Boat',
    6: 'Speed Craft', 7: 'Motorboat', 8: 'Pleasure Yacht',
    9: 'Medium Ferry', 10: 'Large Ferry', 11: 'High Speed Craft'
}

print("YOLO to COCO Converter Initialized")
print(f"Working directory: {os.getcwd()}")
print(f"Data directory: {DATA_DIR}")
print(f"Supporting {len(CLASS_MAPPING)} boats orientation classes")

In [ ]:
def parse_yolo_annotation(txt_file_path):
    """Parse YOLO format annotation file."""
    annotations = []
    if not os.path.exists(txt_file_path):
        return annotations
        
    with open(txt_file_path, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        line = line.strip()
        if line:
            parts = line.split()
            if len(parts) == 5:
                class_id = int(parts[0])
                x_center = float(parts[1])
                y_center = float(parts[2])
                width = float(parts[3])
                height = float(parts[4])
                
                annotations.append({
                    'class_id': class_id,
                    'class_name': CLASS_MAPPING.get(class_id, f"unknown_{class_id}"),
                    'x_center': x_center,
                    'y_center': y_center,
                    'width': width,
                    'height': height
                })
    return annotations

def yolo_to_coco_bbox(x_center, y_center, width, height, img_width, img_height):
    """Convert YOLO normalized coordinates to COCO bbox format."""
    # Convert from normalized to pixel coordinates
    x_center_px = x_center * img_width
    y_center_px = y_center * img_height
    width_px = width * img_width
    height_px = height * img_height
    
    # Convert to COCO format (top-left corner + width, height)
    x_min = x_center_px - width_px / 2
    y_min = y_center_px - height_px / 2
    
    return [x_min, y_min, width_px, height_px]

print("YOLO parsing and COCO conversion functions loaded")

In [ ]:
def create_coco_dataset(image_dir = DATA_DIR, output_file='boat_annotations_coco.json'):
    """Create a complete COCO format dataset from YOLO annotations."""
    
    coco_data = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    # Add categories with supercategories
    for class_id, class_name in CLASS_MAPPING.items():
        vehicle_type = class_name.split('_')[0]  # car, bus, truck, etc.
        coco_data["categories"].append({
            "id": class_id,
            "name": class_name,
            "supercategory": vehicle_type
        })
    
    # Process images and annotations
    jpg_files = glob.glob(os.path.join(image_dir, '*.jpg'))
    annotation_id = 1
    
    print(f"Processing {len(jpg_files)} images...")
    
    for image_id, jpg_file in enumerate(jpg_files, 1):
        try:
            # Get image info
            with Image.open(jpg_file) as img:
                img_width, img_height = img.size
                
            # Add image info
            image_name = os.path.basename(jpg_file)
            coco_data["images"].append({
                "id": image_id,
                "width": img_width,
                "height": img_height,
                "file_name": image_name,
                "license": 1,
                "date_captured": datetime.now().isoformat()
            })
            
            # Process annotations
            txt_file = jpg_file.replace('.jpg', '.txt')
            if os.path.exists(txt_file):
                annotations = parse_yolo_annotation(txt_file)
                
                for ann in annotations:
                    bbox = yolo_to_coco_bbox(
                        ann['x_center'], ann['y_center'], 
                        ann['width'], ann['height'],
                        img_width, img_height
                    )
                    
                    area = bbox[2] * bbox[3]  # width * height
                    
                    coco_data["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": ann['class_id'],
                        "bbox": bbox,
                        "area": area,
                        "iscrowd": 0,
                        "segmentation": []  # Empty for bounding box detection
                    })
                    
                    annotation_id += 1
                    
        except Exception as e:
            print(f"Error processing {jpg_file}: {e}")
            continue
    
    # Save COCO dataset
    with open(output_file, 'w') as f:
        json.dump(coco_data, f, indent=2)
    
    return coco_data

print("COCO dataset creation function ready")

In [ ]:
# Execute COCO conversion
print("Starting YOLO to COCO conversion...")
coco_dataset = create_coco_dataset()

print("\n" + "="*50)
print("COCO CONVERSION COMPLETE")
print("="*50)
print(f"✅ Images processed: {len(coco_dataset['images'])}")
print(f"✅ Annotations created: {len(coco_dataset['annotations'])}")
print(f"✅ Categories defined: {len(coco_dataset['categories'])}")
print(f"✅ Output file: boat_annotations_coco.json")

# Show sample data
if coco_dataset['annotations']:
    sample_annotation = coco_dataset['annotations'][0]
    print(f"\nSample COCO annotation:")
    print(json.dumps(sample_annotation, indent=2))

# Category summary
print(f"\nCategories:")
for cat in coco_dataset['categories']:
    print(f"  {cat['id']}: {cat['name']} (supercategory: {cat['supercategory']})")

In [ ]:
import os
print(f"🔍 Testing DATA_DIR configuration:")
print(f"Current working directory: {os.getcwd()}")
print(f"Data directory (DATA_DIR): {DATA_DIR}")
print(f"Absolute path to data: {os.path.abspath(DATA_DIR)}")

# Check if data directory exists and list contents
if os.path.exists(DATA_DIR):
    print(f"✅ Data directory exists")
    jpg_files = glob.glob(os.path.join(DATA_DIR, '*.jpg'))
    txt_files = glob.glob(os.path.join(DATA_DIR, '*.txt'))
    print(f"📷 Found {len(jpg_files)} .jpg files")
    print(f"📝 Found {len(txt_files)} .txt files")
    
    if jpg_files:
        print(f"📋 Sample images: {[os.path.basename(f) for f in jpg_files[:3]]}")
    if txt_files:
        print(f"📋 Sample annotations: {[os.path.basename(f) for f in txt_files[:3]]}")
else:
    print(f"❌ Data directory does not exist: {os.path.abspath(DATA_DIR)}")
    print(f"💡 Please update DATA_DIR variable to point to your dataset location")

✅ Attribution cell added to notebook!

Attribution content:
  - Original author: Raghav Dharwal
  - Source: Kaggle notebook
  - License: Apache 2.0

## Step 2: Slice with SAHI

In [ ]:
!pip install sahi

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from sahi.slicing import slice_coco
from sahi.utils.file import load_json

run this Python script to slice your images and labels

In [ ]:
coco_dict, coco_path = slice_coco(
    coco_annotation_file_path= "/coco_annotation_file_path",
    image_dir= "/image_dir_path",
    output_coco_annotation_file_name="sliced_coco_test.json",
    output_dir = "/output_dir_path",
    slice_height=640, #set it as you want
    slice_width=640,  #set it as you want
    ignore_negative_samples=False,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
    verbose=True,
)

## Step 3: Convert Sliced COCO back to YOLO
Once SAHI is done, you will have a new folder with sliced images and a new COCO JSON file. Use this script to turn that JSON into the YOLO .txt files you need for training:

In [ ]:
from sahi.utils.coco import Coco

json_path = "/path/to/sliced_coco_test.json"

image_directory = "/path/to/sliced/images/directory"

# Initialize Coco object
coco = Coco.from_coco_dict_or_path(json_path, image_dir=image_directory)

coco.export_as_yolo(
  output_dir="\output\directory\for\yolo\annotations",
  disable_symlink=True
)